In [1]:
from tensorflow import keras
import database_connect as db_connect
import pandas as pd

import numpy as np
import plotting
import xgboost as xgb

In [3]:
xgb_model = xgb.Booster(model_file='xgb_model_final.json')

In [4]:
CONFIG = db_connect.load_config()

In [418]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [5]:
columns = ['id',
    'hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'mon_sin',
 'mon_cos',
 'weekday_sin',
 'weekday_cos',
 'year',
 'RATIO_close_and_MA3',
 'RATIO_close_and_MA6',
 'RATIO_close_and_MA12',
 'RATIO_close_and_MA24',
 'RATIO_close_and_MA48',
 'RATIO_close_and_MA96',
 'RATIO_close_and_MA192',
 'RATIO_close_and_MA384',
 'RATIO_quote_asset_volume_and_MA3',
 'RATIO_quote_asset_volume_and_MA6',
 'RATIO_quote_asset_volume_and_MA12',
 'RATIO_quote_asset_volume_and_MA24',
 'RATIO_quote_asset_volume_and_MA48',
 'RATIO_quote_asset_volume_and_MA96',
 'RATIO_quote_asset_volume_and_MA192',
 'RATIO_quote_asset_volume_and_MA384',
 'RATIO_num_trades_and_MA3',
 'RATIO_num_trades_and_MA6',
 'RATIO_num_trades_and_MA12',
 'RATIO_num_trades_and_MA24',
 'RATIO_num_trades_and_MA48',
 'RATIO_num_trades_and_MA96',
 'RATIO_num_trades_and_MA192',
 'RATIO_num_trades_and_MA384',
 'RATIO_close_and_STD3',
 'RATIO_close_and_STD6',
 'RATIO_close_and_STD12',
 'RATIO_close_and_STD24',
 'RATIO_close_and_STD48',
 'RATIO_close_and_STD96',
 'RATIO_close_and_STD192',
 'RATIO_close_and_STD384',
 'RSI_3_close',
 'RSI_6_close',
 'RSI_12_close',
 'RSI_24_close',
 'RSI_48_close',
 'RSI_96_close',
 'RSI_192_close',
 'RSI_384_close',
 'RSI_3_quote_asset_volume',
 'RSI_6_quote_asset_volume',
    'RSI_12_quote_asset_volume', 'RSI_24_quote_asset_volume',
    'RSI_48_quote_asset_volume', 'RSI_96_quote_asset_volume',
    'RSI_192_quote_asset_volume', 'RSI_384_quote_asset_volume',
    'RSI_3_num_trades', 'RSI_6_num_trades', 'RSI_12_num_trades',
    'RSI_24_num_trades', 'RSI_48_num_trades', 'RSI_96_num_trades',
    'RSI_192_num_trades', 'RSI_384_num_trades' ,
 'predicted']

In [420]:
model = keras.models.load_model('models/2024-03-03 01-00-55 val_PR 0.518 Dense.hd5', compile=False)

In [421]:
TIME_STEP = 3

In [422]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 65)]              0         
                                                                 
 normalization_1 (Normaliza  (None, 65)                131       
 tion)                                                           
                                                                 
 dense_4 (Dense)             (None, 136)               8976      
                                                                 
 dense_5 (Dense)             (None, 136)               18632     
                                                                 
 dense_6 (Dense)             (None, 65)                8905      
                                                                 
 dense_7 (Dense)             (None, 16)                1056      
                                                           

In [6]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [7]:
# sql = 'select min(timestamp) from trading_bot_historydataeth \
#         where predicted is null and hour_sin is not null \
#         limit 1'

sql = 'select min(timestamp) from trading_bot_historydataeth \
        where hour_sin is not null \
        limit 1'


In [8]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchone()

In [9]:
result = result[0]

In [10]:
result_str = result.strftime('%Y-%m-%d %H:%M:%S')
result_str

'2023-12-14 20:00:00'

In [11]:
sql = 'SELECT "'
sql += '", "'.join(columns)
sql += '"'

In [12]:
sql

'SELECT "id", "hour_sin", "hour_cos", "day_sin", "day_cos", "mon_sin", "mon_cos", "weekday_sin", "weekday_cos", "year", "RATIO_close_and_MA3", "RATIO_close_and_MA6", "RATIO_close_and_MA12", "RATIO_close_and_MA24", "RATIO_close_and_MA48", "RATIO_close_and_MA96", "RATIO_close_and_MA192", "RATIO_close_and_MA384", "RATIO_quote_asset_volume_and_MA3", "RATIO_quote_asset_volume_and_MA6", "RATIO_quote_asset_volume_and_MA12", "RATIO_quote_asset_volume_and_MA24", "RATIO_quote_asset_volume_and_MA48", "RATIO_quote_asset_volume_and_MA96", "RATIO_quote_asset_volume_and_MA192", "RATIO_quote_asset_volume_and_MA384", "RATIO_num_trades_and_MA3", "RATIO_num_trades_and_MA6", "RATIO_num_trades_and_MA12", "RATIO_num_trades_and_MA24", "RATIO_num_trades_and_MA48", "RATIO_num_trades_and_MA96", "RATIO_num_trades_and_MA192", "RATIO_num_trades_and_MA384", "RATIO_close_and_STD3", "RATIO_close_and_STD6", "RATIO_close_and_STD12", "RATIO_close_and_STD24", "RATIO_close_and_STD48", "RATIO_close_and_STD96", "RATIO_close

In [13]:
sql = sql + f" FROM trading_bot_historydataeth \
        WHERE timestamp >= '{result_str}' \
        ORDER BY timestamp;"

In [14]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
conn.close()

In [15]:
df_ready_for_pred = pd.DataFrame(result, columns=columns)

In [16]:
df_ready_for_pred.head()

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades,predicted
0,37580,-0.866025,0.500000,2.079117e-01,-0.978148,-2.449294e-16,1.0,0.433884,-0.900969,2023,...,49.942776,0.000000,27.183127,45.828688,45.221607,49.166098,49.793056,49.910483,49.980198,W
1,37581,-0.707107,0.707107,2.079117e-01,-0.978148,-2.449294e-16,1.0,0.433884,-0.900969,2023,...,49.951091,0.000000,32.646250,46.459504,47.343045,49.025131,49.861485,49.697697,49.984556,W
2,37582,-0.500000,0.866025,2.079117e-01,-0.978148,-2.449294e-16,1.0,0.433884,-0.900969,2023,...,50.064776,88.139033,44.713062,47.146063,45.848158,50.337329,50.407040,49.839363,50.100936,W
3,37583,-0.258819,0.965926,2.079117e-01,-0.978148,-2.449294e-16,1.0,0.433884,-0.900969,2023,...,50.228667,99.067539,31.295772,48.485934,51.945474,50.518233,50.387212,50.228037,50.246409,W
4,37584,0.000000,1.000000,1.224647e-16,-1.000000,-2.449294e-16,1.0,-0.433884,-0.900969,2023,...,50.015306,60.584103,40.949744,44.021697,49.677811,49.928268,50.177326,50.094393,50.057948,W


In [17]:
df_ready_for_pred.shape

(4980, 67)

In [18]:
df_ready_for_pred[['predicted', 'id']].groupby('predicted').count()

,id
predicted,
B,26
H,562
S,61
W,1246


In [436]:
tensor = dataframe_to_tensor(df_ready_for_pred.drop (columns= ['id', 'predicted']), TIME_STEP)

In [437]:
tensor.shape

(1893, 3, 65)

In [19]:
# predictions = model.predict(df_ready_for_pred.drop (columns= ['id', 'predicted']))
# predictions = model.predict(tensor)


Dmatrix = xgb.DMatrix(data=df_ready_for_pred.drop (columns= ['id', 'predicted']))
predictions = xgb_model.predict(Dmatrix)

In [20]:
predictions.shape

(4980, 4)

In [21]:
predictions_df = pd.DataFrame(predictions, columns=['B', 'H', 'S', 'W'])

In [22]:
predictions_df['Final'] = predictions_df.idxmax(axis=1)

In [23]:
predictions_df.head()

,B,H,S,W,Final
0,0.000417,0.008189,0.000040,0.991354,W
1,0.000023,0.004749,0.000247,0.994980,W
2,0.000005,0.003966,0.000329,0.995700,W
3,0.000017,0.003317,0.000415,0.996251,W
4,0.000270,0.001808,0.000008,0.997914,W


In [24]:
predictions_df.groupby('Final').count()

,B,H,S,W
Final,,,,
B,80,80,80,80
H,1325,1325,1325,1325
S,207,207,207,207
W,3368,3368,3368,3368


In [25]:
df_ready_for_pred['predicted'] = predictions_df['Final']

In [26]:
df_ready_for_pred['predicted'].tail()

4975    S
4976    H
4977    H
4978    H
4979    W
Name: predicted, dtype: object

In [27]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [28]:
for index, row in df_ready_for_pred.iterrows():
    sql = f"UPDATE trading_bot_historydataeth \
            SET predicted = '{row['predicted']}' \
            WHERE id = {row['id']};"
    with conn.cursor() as cursor:
        cursor.execute(sql)
    conn.commit()
conn.close()

In [29]:
conn = db_connect.connect(CONFIG)
sql = "SELECT * FROM trading_bot_historydataeth \
        WHERE predicted is not null  AND timestamp >='2024-01-09' \
        ORDER BY timestamp ASC;"
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
conn.close()

Connected to the PostgreSQL server.


In [30]:
conn = db_connect.connect(CONFIG)
sql = "SELECT COLUMN_NAME \
    FROM INFORMATION_SCHEMA.COLUMNS \
    WHERE TABLE_NAME = 'trading_bot_historydataeth' \
    ORDER BY ORDINAL_POSITION;"
with conn.cursor() as cursor:
    cursor.execute(sql)
    col_names = cursor.fetchall()
conn.close()

Connected to the PostgreSQL server.


In [31]:
cols_list = []
for col in col_names:
    cols_list.append(col[0])

In [32]:
result_df = pd.DataFrame(result, columns=cols_list)

In [33]:
result_df

,id,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,...,RSI_384_num_trades,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_3_quote_asset_volume,RSI_48_num_trades,RSI_48_quote_asset_volume,RSI_6_num_trades,RSI_6_quote_asset_volume,RSI_96_num_trades,RSI_96_quote_asset_volume
0,38184,2024-01-09 00:00:00+05:00,2330.43,2336.70,2289.98,2306.90,33340.2527,1704760000000,7.710257e+07,66482,...,50.417838,50.422666,83.038987,82.683290,54.036062,53.700114,35.900702,31.624271,52.154077,51.832814
1,38185,2024-01-09 01:00:00+05:00,2306.89,2315.18,2296.56,2309.36,19092.9739,1704770000000,4.403277e+07,34435,...,50.025266,50.072409,59.635997,63.537811,51.522813,51.698179,34.895273,32.886977,48.509641,48.551849
2,38186,2024-01-09 02:00:00+05:00,2309.35,2316.60,2306.00,2312.10,9560.0333,1704770000000,2.210319e+07,19945,...,49.819697,49.809219,45.509578,43.602966,50.311648,50.426330,41.114613,40.231972,49.301729,49.222913
3,38187,2024-01-09 03:00:00+05:00,2312.09,2314.69,2307.02,2309.13,9382.4544,1704770000000,2.168060e+07,20175,...,49.932173,49.917523,0.491800,0.000000,50.426042,50.571026,45.842406,46.057880,50.048528,50.061796
4,38188,2024-01-09 04:00:00+05:00,2309.12,2318.00,2300.40,2315.20,8726.1326,1704780000000,2.015283e+07,21479,...,49.903714,49.923296,9.573140,0.000000,50.584997,50.438101,51.228791,50.295265,49.856044,49.657068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,43245,2024-07-09 03:00:00+05:00,3074.08,3074.31,3043.80,3064.45,10660.1096,1720497599999,3.262292e+07,47546,...,50.246413,50.117304,51.032229,55.930384,50.503626,50.467473,47.296866,43.766663,42.860158,42.225325
4372,43246,2024-07-09 04:00:00+05:00,3064.45,3085.76,3058.72,3070.54,12446.4072,1720501199999,3.824143e+07,56069,...,50.314735,50.166968,12.813266,14.381614,50.739195,50.527630,56.329877,58.575122,45.365593,43.319431
4373,43247,2024-07-09 05:00:00+05:00,3070.54,3074.80,3054.14,3068.08,5591.3537,1720504799999,1.714266e+07,46490,...,50.263778,50.045054,25.687161,20.444866,50.638276,49.701338,57.905786,52.540927,47.499418,47.017343
4374,43248,2024-07-09 06:00:00+05:00,3068.08,3090.38,3061.60,3074.69,9534.7657,1720508399999,2.936728e+07,63592,...,50.360928,50.148230,72.790024,45.819874,51.496140,50.692587,56.017756,52.501995,48.279932,47.627847


In [34]:
plotting.chart_with_markers(result_df, 'predicted')

c:\Users\user\Documents\GitHub\BinanceTradingBot\plotting.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sell['close'] = df_sell['close'].apply(lambda x: x*1.05)
c:\Users\user\Documents\GitHub\BinanceTradingBot\plotting.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buy['close'] = df_buy['close'].apply(lambda x: x*0.95)
